In [1]:
#| default_exp models.tsmixer

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| export

import torch
import torch.nn as nn
import torch.nn.functional as F

from neuralforecast.losses.pytorch import MAE
from neuralforecast.common._base_multivariate import BaseMultivariate

In [58]:
#| export
class TSMixer(BaseMultivariate):
    """ TSMixer

    TODO documentation here

    **Parameters:**<br>
    `h`: int, Forecast horizon. <br>
    `input_size`: int, autorregresive inputs size, y=[1,2,3,4] input_size=2 -> y_[t-2:t]=[1,2].<br>
    `n_series`: int, number of time-series.<br>
    `stat_exog_list`: str list, static exogenous columns.<br>
    `hist_exog_list`: str list, historic exogenous columns.<br>
    `futr_exog_list`: str list, future exogenous columns.<br>
    `n_stacks`: int=2, number of stacks in the model.<br>
    `multi_layer`: int=5, multiplier for FC hidden size on StemGNN blocks.<br>
    `dropout_rate`: float=0.5, dropout rate.<br>
    `leaky_rate`: float=0.2, alpha for LeakyReLU layer on Latent Correlation layer.<br>
    `loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
    `max_steps`: int=1000, maximum number of training steps.<br>
    `learning_rate`: float=1e-3, Learning rate between (0, 1).<br>
    `num_lr_decays`: int=-1, Number of learning rate decays, evenly distributed across max_steps.<br>
    `early_stop_patience_steps`: int=-1, Number of validation iterations before early stopping.<br>
    `val_check_steps`: int=100, Number of training steps between every validation loss check.<br>
    `batch_size`: int, number of windows in each batch.<br>
    `step_size`: int=1, step size between each window of temporal data.<br>
    `scaler_type`: str='robust', type of scaler for temporal inputs normalization see [temporal scalers](https://nixtla.github.io/neuralforecast/common.scalers.html).<br>
    `random_seed`: int, random_seed for pytorch initializer and numpy generators.<br>
    `num_workers_loader`: int=os.cpu_count(), workers to be used by `TimeSeriesDataLoader`.<br>
    `drop_last_loader`: bool=False, if True `TimeSeriesDataLoader` drops last non-full batch.<br>
    `alias`: str, optional,  Custom name of the model.<br>
    `**trainer_kwargs`: int,  keyword trainer arguments inherited from [PyTorch Lighning's trainer](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.trainer.trainer.Trainer.html?highlight=trainer).<br>    
    """
    # Class attributes
    SAMPLING_TYPE = 'multivariate'
    
    def __init__(self,
                 h,
                 input_size,
                 n_series,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 n_stacks = 2,
                 multi_layer: int = 5,
                 dropout_rate: float = 0.5,
                 leaky_rate: float = 0.2,
                 loss = MAE(),
                 valid_loss = None,
                 max_steps: int = 1000,
                 learning_rate: float = 1e-3,
                 num_lr_decays: int = 3,
                 early_stop_patience_steps: int =-1,
                 val_check_steps: int = 100,
                 batch_size: int = 32,
                 step_size: int = 1,
                 scaler_type: str = 'robust',
                 random_seed: int = 1,
                 num_workers_loader = 0,
                 drop_last_loader = False,
                 **trainer_kwargs):

        # Inherit BaseMultivariate class
        super(TSMixer, self).__init__(h=h,
                                      input_size=input_size,
                                      n_series=n_series,
                                      futr_exog_list=futr_exog_list,
                                      hist_exog_list=hist_exog_list,
                                      stat_exog_list=stat_exog_list,                                    
                                      loss=loss,
                                      valid_loss=valid_loss,
                                      max_steps=max_steps,
                                      learning_rate=learning_rate,
                                      num_lr_decays=num_lr_decays,
                                      early_stop_patience_steps=early_stop_patience_steps,
                                      val_check_steps=val_check_steps,
                                      batch_size=batch_size,
                                      step_size=step_size,
                                      scaler_type=scaler_type,
                                      num_workers_loader=num_workers_loader,
                                      drop_last_loader=drop_last_loader,
                                      random_seed=random_seed,
                                      **trainer_kwargs)


    def res_block(self, inputs, batch_size, input_size, num_channels):
        self.batch_norm = nn.BatchNorm2d(num_features=num_channels)
        self.linear = nn.Linear(in_features=input_size, out_features=input_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
    
        # time block
        x = nn.BatchNorm2d(num_features=num_channels)
        x = torch.transpose(x, 3, 2)
        x = nn.Linear(in_features=num_channels, out_features=num_channels)(x)
        x = nn.ReLU(x)
        x = nn.Dropout(0.1)(x)
        res = inputs + x

        # feature block
        x = torch.transpose(res, 3, 2)
        x = nn.BatchNorm2d(num_features=num_channels)(x)
        x = nn.Linear(in_features=num_channels, out_features=num_channels)(x)
        x = nn.ReLU(x)
        x = nn.Dropout(0.1)(x)
        x = nn.Linear(in_features=num_channels, out_features=num_channels)(x)
        x = nn.Dropout(0.1)(x)
        return x + res

    def forward(self, windows_batch):
        inputs = windows_batch['insample_y']
        batch_size = windows_batch.shape[0]
        input_size = windows_batch.shape[1]
        num_channels = windows_batch.shape[2]
        horizon = self.h

        layers = nn.ModuleList(
            [# TODO loop this based on `num_stacks`
             self.res_block(inputs=inputs,
                            batch_size=batch_size,
                            input_size=input_size,
                            num_channels=num_channels),
             torch.transpose(3, 2),
             nn.Linear(in_features=input_size, out_features=horizon),
             torch.transpose(3, 2)]
            )
        
        for i, l in enumerate(layers):
            if i == 0:
                x = l(inputs)
            else:
                x = (l(x))

        return x


In [59]:
#| eval: false
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.pytorch import MAE

Y_train_df = AirPassengersPanel[AirPassengersPanel.ds<AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 132 train
Y_test_df = AirPassengersPanel[AirPassengersPanel.ds>=AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 12 test

model = TSMixer(h=12,
                input_size=24,
                n_series=2,
                stat_exog_list=['airline1'],
                futr_exog_list=['trend'],
                scaler_type='robust',
                max_steps=200,
                early_stop_patience_steps=-1,
                val_check_steps=10,
                learning_rate=1e-3,
                loss=None,
                valid_loss=None,
                batch_size=32
                )

list(model.parameters())

fcst = NeuralForecast(models=[model], freq='M')
fcst.fit(df=Y_train_df, static_df=AirPassengersStatic, val_size=12)
# forecasts = fcst.predict(futr_df=Y_test_df)

Seed set to 1


ValueError: optimizer got an empty parameter list

In [13]:
list(model.parameters())

[]

In [34]:
class NeuralNetwork(BaseMultivariate):
        # Class attributes
    SAMPLING_TYPE = 'multivariate'
    
    def __init__(self,
                 C,
                 H,
                 T,
                 inputs,
                 h,
                 input_size,
                 n_series,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 n_stacks = 2,
                 multi_layer: int = 5,
                 dropout_rate: float = 0.5,
                 leaky_rate: float = 0.2,
                 loss = MAE(),
                 valid_loss = None,
                 max_steps: int = 1000,
                 learning_rate: float = 1e-3,
                 num_lr_decays: int = 3,
                 early_stop_patience_steps: int =-1,
                 val_check_steps: int = 100,
                 batch_size: int = 32,
                 step_size: int = 1,
                 scaler_type: str = 'robust',
                 random_seed: int = 1,
                 num_workers_loader = 0,
                 drop_last_loader = False,
                 **trainer_kwargs):

        # Inherit BaseMultivariate class
        super(TSMixer, self).__init__(h=h,
                                      input_size=input_size,
                                      n_series=n_series,
                                      futr_exog_list=futr_exog_list,
                                      hist_exog_list=hist_exog_list,
                                      stat_exog_list=stat_exog_list,                                    
                                      loss=loss,
                                      valid_loss=valid_loss,
                                      max_steps=max_steps,
                                      learning_rate=learning_rate,
                                      num_lr_decays=num_lr_decays,
                                      early_stop_patience_steps=early_stop_patience_steps,
                                      val_check_steps=val_check_steps,
                                      batch_size=batch_size,
                                      step_size=step_size,
                                      scaler_type=scaler_type,
                                      num_workers_loader=num_workers_loader,
                                      drop_last_loader=drop_last_loader,
                                      random_seed=random_seed,
                                      **trainer_kwargs)

        self.inputs = inputs
        self.C = C
        self.H = H
        self.T = T

        self.batch_norm = nn.BatchNorm2d(num_features=C)
        self.linear = nn.Linear(in_features=H, out_features=H)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        inputs = self.inputs
        H = self.H
        C = self.C
        T = self.T

        # time block
        x = self.batch_norm()
        x = torch.transpose(x, 3, 2)
        x = self.linear(x)
        x = self.relu(x)
        x = self.dropout(x)
        res = inputs + x

        # feature block
        x = torch.transpose(res, 3, 2)
        x = self.batch_norm(x)
        x = self.linear(x)
        x = self.relu(x)
        x = self.dropout(0.1)(x)
        x = self.linear(x)
        x = self.dropout(x)
        x = x + res

        # temporal projection
        x = torch.transpose(res, 3, 2)
        x = nn.Linear(in_features=H, out_features=T)(x)
        x = torch.transpose(x, 3, 2)
        return x

shape = (1, 1, 365, 1) # (batch_size, channels, height, width) for unnivariate data
shape = (1, 2, 365, 1) # (batch_size, channels, height, width) for multi variate data
inputs = torch.rand(shape)

N = inputs.shape[0]
C = inputs.shape[1]
H = inputs.shape[2]
W = inputs.shape[3]
T = 90
m = NeuralNetwork(inputs=inputs, C=C, H=H, T=T, h=12,
                 input_size=24,
                 n_series=2,)
list(m.parameters())

TypeError: super(type, obj): obj must be an instance or subtype of type

In [39]:
class NeuralNetwork(nn.Module):
    def __init__(self, inputs, C, H, T):
        super().__init__()
        self.C = C
        self.H = H
        self.T = T
        self.inputs = inputs

        self.batch_norm = nn.BatchNorm2d(num_features=C)
        self.linear = nn.Linear(in_features=H, out_features=H)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        H = self.H
        C = self.C
        T = self.T
        inputs = self.inputs

        # time block
        x = self.batch_norm()
        x = torch.transpose(x, 3, 2)
        x = self.linear(x)
        x = self.relu(x)
        x = self.dropout(x)
        res = inputs + x

        # feature block
        x = torch.transpose(res, 3, 2)
        x = self.batch_norm(x)
        x = self.linear(x)
        x = self.relu(x)
        x = self.dropout(0.1)(x)
        x = self.linear(x)
        x = self.dropout(x)
        x = x + res

        # temporal projection
        x = torch.transpose(res, 3, 2)
        x = nn.Linear(in_features=H, out_features=T)(x)
        x = torch.transpose(x, 3, 2)
        return x

N = inputs.shape[0]
C = inputs.shape[1]
H = inputs.shape[2]
W = inputs.shape[3]
T = 90
m = NeuralNetwork(inputs=inputs, C=C, H=H, T=T)
list(m.parameters())

[Parameter containing:
 tensor([1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0.], requires_grad=True),
 Parameter containing:
 tensor([[-0.0076,  0.0512,  0.0308,  ...,  0.0056,  0.0078,  0.0049],
         [ 0.0094,  0.0163, -0.0493,  ...,  0.0114,  0.0445,  0.0011],
         [-0.0361, -0.0072,  0.0217,  ...,  0.0083,  0.0368, -0.0186],
         ...,
         [-0.0474, -0.0363,  0.0483,  ...,  0.0089,  0.0008,  0.0405],
         [ 0.0172,  0.0100,  0.0167,  ..., -0.0243, -0.0012,  0.0497],
         [ 0.0407, -0.0343,  0.0194,  ...,  0.0160, -0.0240, -0.0064]],
        requires_grad=True),
 Parameter containing:
 tensor([ 1.6142e-02, -2.8519e-02,  2.0091e-02, -3.0201e-02,  1.0083e-02,
         -3.6561e-03, -3.5331e-02, -3.7638e-02, -6.0492e-03,  2.5079e-02,
          5.0956e-02, -4.9123e-02,  4.1784e-02,  1.7217e-02,  2.8004e-02,
         -1.9967e-02,  1.4762e-02, -1.1958e-02,  9.2816e-03,  3.1025e-02,
          7.4955e-03,  4.2849e-02,  3.3125e-02, -2.2864e-02,  3.

## take 2

In [165]:
#| export
class TSMixer(BaseMultivariate):
    # Class attributes
    SAMPLING_TYPE = 'multivariate'
    
    def __init__(self,
                 h,
                 input_size,
                 n_series,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 n_stacks = 2,
                 multi_layer: int = 5,
                 dropout_rate: float = 0.5,
                 leaky_rate: float = 0.2,
                 loss = MAE(),
                 valid_loss = None,
                 max_steps: int = 1000,
                 learning_rate: float = 1e-3,
                 num_lr_decays: int = 3,
                 early_stop_patience_steps: int =-1,
                 val_check_steps: int = 100,
                 batch_size: int = 32,
                 step_size: int = 1,
                 scaler_type: str = 'robust',
                 random_seed: int = 1,
                 num_workers_loader = 0,
                 drop_last_loader = False,
                 **trainer_kwargs):

        # Inherit BaseMultivariate class
        super(TSMixer, self).__init__(h=h,
                                      input_size=input_size,
                                      n_series=n_series,
                                      futr_exog_list=futr_exog_list,
                                      hist_exog_list=hist_exog_list,
                                      stat_exog_list=stat_exog_list,                                    
                                      loss=loss,
                                      valid_loss=valid_loss,
                                      max_steps=max_steps,
                                      learning_rate=learning_rate,
                                      num_lr_decays=num_lr_decays,
                                      early_stop_patience_steps=early_stop_patience_steps,
                                      val_check_steps=val_check_steps,
                                      batch_size=batch_size,
                                      step_size=step_size,
                                      scaler_type=scaler_type,
                                      num_workers_loader=num_workers_loader,
                                      drop_last_loader=drop_last_loader,
                                      random_seed=random_seed,
                                      **trainer_kwargs)

        # Exogenous variables
        self.futr_input_size = len(self.futr_exog_list)
        self.hist_input_size = len(self.hist_exog_list)
        self.stat_input_size = len(self.stat_exog_list)

        self.unit = n_series
        self.stack_cnt = n_stacks
        self.alpha = leaky_rate
        self.time_step = input_size
        self.horizon = h
        self.dropout_rate = dropout_rate
        self.input_size = input_size
        self.num_channels = n_series
        self.H = H

        self.batch_norm = nn.BatchNorm2d(num_features=self.num_channels)
        self.linear = nn.Linear(in_features=H, out_features=H)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, windows_batch, *args, **kwargs):
        inputs = windows_batch['insample_y']
        print('raw', inputs.shape)
        
        x = inputs.unsqueeze(1)#.permute(0, 1, 2, 3)
        print('unsqueeze', x.shape)
        
        for _ in range(0, 1): # self.stack_cnt
            # time block
            # x = nn.BatchNorm2d(num_features=self.num_channels)(x)
            x = torch.transpose(x, 3, 2)
            print('transpose', x.shape)
            
            x = nn.Linear(in_features=self.input_size, out_features=self.input_size)(x)
            print('linear', x.shape)
            
            x = nn.ReLU()(x)
            print('relu', x.shape)

            x = nn.Dropout(p=self.dropout_rate)(x)
            print('dropout', x.shape)

            res = inputs.unsqueeze(1) + x
            print('res', res.shape)
            print('\n')
            print('--------')

            # feature block
            x = torch.transpose(res, 3, 2)
            print('feature transpose', x.shape)

            x = nn.BatchNorm2d(num_features=self.num_channels)(x)
            print('batch norm', x.shape)

            x = nn.Linear(in_features=self.input_size, out_features=self.input_size)(x)
            print('linear', x.shape)

            x = nn.ReLU()(x)
            print('relu', x.shape)

            x = nn.Dropout(p=self.dropout_rate)(x)
            print('dropout', x.shape)

            x = nn.Linear(in_features=self.input_size, out_features=self.input_size)(x)
            print('linear', x.shape)

            x = nn.Dropout(p=self.dropout_rate)(x)
            print('dropout', x.shape)

            x = x + res
            print('final x', x.shape)

        # temporal projection
        x = torch.transpose(res, 3, 2)
        x = nn.Linear(in_features=self.input_size, out_features=self.horizon)(x)
        x = torch.transpose(x, 3, 2)
        return x

In [166]:
model = TSMixer(h=12,
                input_size=24,
                n_series=2,
                stat_exog_list=['airline1'],
                futr_exog_list=['trend'],
                scaler_type='robust',
                max_steps=200,
                early_stop_patience_steps=-1,
                val_check_steps=10,
                learning_rate=1e-3,
                loss=None,
                valid_loss=None,
                batch_size=32
                )

list(model.parameters())

fcst = NeuralForecast(models=[model], freq='M')
fcst.fit(df=Y_train_df, static_df=AirPassengersStatic, val_size=12)
forecasts = fcst.predict(futr_df=Y_test_df)

Seed set to 1


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

raw torch.Size([1, 24, 2])
unsqueeze torch.Size([1, 1, 24, 2])
transpose torch.Size([1, 1, 2, 24])
linear torch.Size([1, 1, 2, 24])
relu torch.Size([1, 1, 2, 24])
dropout torch.Size([1, 1, 2, 24])


RuntimeError: The size of tensor a (2) must match the size of tensor b (24) at non-singleton dimension 3

In [99]:
list(model.parameters())

[Parameter containing:
 tensor([1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0.], requires_grad=True),
 Parameter containing:
 tensor([[ 2.6970e-02, -2.3103e-02, -1.0147e-02,  ...,  3.1881e-02,
          -7.6078e-03,  2.1505e-02],
         [ 2.5764e-03, -4.8852e-02, -2.5679e-02,  ..., -2.0233e-02,
           3.5104e-03, -2.0108e-02],
         [ 4.9278e-02,  5.0543e-02, -6.0997e-03,  ..., -3.9860e-02,
          -3.8550e-02,  4.2704e-02],
         ...,
         [ 1.2923e-02,  5.0932e-02,  1.9553e-02,  ..., -1.0130e-02,
           4.5885e-02, -4.2174e-02],
         [ 2.2950e-02, -9.2104e-03, -2.2901e-02,  ...,  2.8679e-03,
          -3.2190e-02,  4.1641e-03],
         [ 4.4899e-02,  8.1050e-03, -2.6598e-02,  ...,  1.6064e-02,
          -3.6001e-02,  1.9062e-05]], requires_grad=True),
 Parameter containing:
 tensor([-4.2905e-02, -2.3027e-02,  2.1914e-02,  2.3291e-02,  3.6993e-02,
          4.3965e-02,  3.8235e-02, -8.7991e-03, -4.0697e-02,  4.9842e-02,
          1.3931